In [18]:
import time

In [19]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common import exceptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [20]:
wd = "C:\\Users\\user\\Downloads\\chromedriver_win32\\chromedriver"
addr = "https://entertain.naver.com/read?oid=109&aid=0004166734"

In [21]:
driver = webdriver.Chrome(wd)
driver.get(addr)

In [22]:
pages = 0

In [6]:
try:
    while True:
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"#cbox_module > div > div.u_cbox_paginate > a"))).click()
        time.sleep(1.5)
        print(pages, end=" ")
        pages+=1
except exceptions.ElementNotVisibleException as e: # 페이지 끝
    pass
    
except Exception as e: # 다른 예외 발생시 확인
    print(e)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 Message: element click intercepted: Element <a href="#" class="u_cbox_btn_more" data-action="page#more" data-log="RPC.more">...</a> is not clickable at point (380, 363). Other element would receive the click: <div id="footer">...</div>
  (Session info: chrome=80.0.3987.106)



In [23]:
html = driver.page_source
dom = BeautifulSoup(html, "lxml")

In [24]:
# 댓글이 들어있는 페이지 전체 크롤링
comments_raw = dom.find_all("span", {"class" : "u_cbox_contents"})
# 댓글의 text만 뽑는다.
comments = [comment.text for comment in comments_raw]

In [25]:
comments[:3]

['ㅋㅋㅋㅋㅋ이 4명 케미 왜이리 웃김 ㅋㅋㅋㅋ 수다만 떨었는데도 빵빵터짐 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ',
 '의미부여없이 웃기기로는 놀면뭐하니에서 오늘이 젤웃겼음ㅋㅋ지석진 조세호냅두고선 문닫고도망치는거 최고의 킬링포인트ㅋㅋㅋㅋ',
 '유느 행복해하는거 넘 보기좋네요ㅎㅎㅎ']

In [10]:
p_comment_list = []

In [11]:
for comment in comments:
    p_comment_list.append(comment)

In [264]:
file = open('p_comment36.txt','w',encoding = 'utf-8')
for comment in p_comment_list:
    file.write(comment + '\n')
file.close()